In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../../.env")

import pandas as pd

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
os.environ.clear()
load_dotenv("../.env", override=True) 
print(os.environ["ANTHROPIC_API_KEY"])